# Titanic - Machine Learning from Disaster

[https://www.kaggle.com/c/titanic](https://www.kaggle.com/c/titanic)

Author: Vaasudevan Srinivasan [(Portfolio)](https://vaasudevans.github.io) <br>
Created on: July 06, 2021 <br>

In [1]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd

In [2]:
train_df = pd.read_csv('/kaggle/input/titanic/train.csv')
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# Split 10% for validation
x_df = train_df.drop('Survived', axis=1)
y_df = train_df['Survived']
X_train, X_val, y_train, y_val = train_test_split(x_df, y_df, test_size=0.1, random_state=42)
print(X_train.shape, X_val.shape)

(801, 11) (90, 11)


# Pipelines for preprocessing

In [4]:
# https://datascience.stackexchange.com/a/81559
# https://stackoverflow.com/a/62234209

embark_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('one-hot', OneHotEncoder())
])

age_fare_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='median')),
    ('std', StandardScaler())
])

ct = ColumnTransformer([
    ('impute-onehot', embark_pipeline, ['Embarked']),
    ('impute-std', age_fare_pipeline, ['Age', 'Fare']),
    ('onehot', OneHotEncoder(), ['Pclass', 'Sex'])
])

X_train = ct.fit_transform(X_train)
X_val = ct.fit_transform(X_val)

# Training and Validation

In [5]:
rf = RandomForestClassifier(random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
rf.score(X_val, y_val)

0.7444444444444445

In [6]:
lr = LogisticRegression(random_state=42, n_jobs=-1)
lr.fit(X_train, y_train)
lr.score(X_val, y_val)

0.8333333333333334

# Prediction

In [7]:
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [8]:
# Predict using Logistic Regression
X_test = ct.fit_transform(test_df)
test_df['Survived'] = lr.predict(X_test)
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1


In [9]:
test_df[['PassengerId', 'Survived']].to_csv('submission.csv', index=False)